In [1]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd

In [2]:
db = database('trfc.db')

In [3]:
app, rt = fast_app()

In [4]:
server = JupyUvi(app)

In [5]:
data = {
    'min_season': 1921,
    'max_season': 2024
}

venues = ('H', 'A', 'N')

min_games = 10

In [6]:
focus = 'm.manager_name'
focus = 'r.season'
focus = 'r.opposition'

In [9]:
query = f'''
    SELECT 
            {focus},
            COUNT(*) as P,
            COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) as W,
            COUNT(CASE WHEN r.outcome = 'D' THEN 1 END) as D,
            COUNT(CASE WHEN r.outcome = 'L' THEN 1 END) as L,
            SUM(r.goals_for) as GF,
            SUM(r.goals_against) as GA,
            SUM(r.goals_for) - SUM(r.goals_against) as GD,
            ROUND(CAST(COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) AS FLOAT) / COUNT(*) * 100, 1) as win_pc
        FROM results r
        LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
            AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
        LEFT JOIN managers m ON mr.manager_id = m.manager_id
        LEFT JOIN league_tiers lt ON r.season = lt.season AND r.game_type = 'League'
        WHERE CAST(SUBSTRING(r.season, 1, 4) AS INTEGER) >= {data['min_season']} AND CAST(SUBSTRING(r.season, 1, 4) AS INTEGER) <= {data['max_season']}
            AND r.venue IN ('H', 'A', 'N')
        GROUP BY {focus}
        HAVING COUNT(*) >= {min_games}
        ORDER BY P DESC
'''

In [10]:
df = pd.read_sql(query, db.conn)

df.query("opposition=='Barnsley'")

,opposition,P,W,D,L,GF,GA,GD,win_pc
29,Barnsley,56,14,18,24,71.0,87.0,-16.0,25.0


In [18]:
# server.stop()